# HW4 Trees and Forests

Official instructions:

https://www.cs.tufts.edu/cs/135/2023f/hw4.html

This is the *starter code* notebook for Problems 1 and 2, looking at sentiment classification using bag-of-words features of product reviews.

In [1]:
import numpy as np
import pandas as pd
import os
import sys
import time


In [2]:
import sklearn.tree
import sklearn.linear_model
import sklearn.metrics
import sklearn.ensemble
import sklearn.model_selection


In [3]:
# From the HW4 starter code
from pretty_print_sklearn_tree import pretty_print_sklearn_tree


In [4]:
# Plotting utils
import matplotlib
import matplotlib.pyplot as plt

plt.style.use('seaborn-v0_8') # pretty matplotlib plots

import seaborn as sns
sns.set('notebook', font_scale=1.25, style='whitegrid')


# Load all data from train/valid/test

In [5]:
DATA_DIR = os.path.join("data_product_reviews/")


### Load training

In [6]:
x_tr_df = pd.read_csv(os.path.join(DATA_DIR, 'x_train.csv.zip'))
y_tr_df = pd.read_csv(os.path.join(DATA_DIR, 'y_train.csv'))
x_tr_NF = np.minimum(x_tr_df.values, 1.0).copy()
y_tr_N = y_tr_df.values[:,0].copy()

print("Training data")
print("x_tr_NF.shape: %s" % str(x_tr_NF.shape))
print("y_tr_N.shape : %s" % str(y_tr_N.shape))
print("mean(y_tr_N) : %.3f" % np.mean(y_tr_N))


Training data
x_tr_NF.shape: (6346, 7729)
y_tr_N.shape : (6346,)
mean(y_tr_N) : 0.500


### Load validation set

In [7]:
x_va_df = pd.read_csv(os.path.join(DATA_DIR, 'x_valid.csv.zip'))
y_va_df = pd.read_csv(os.path.join(DATA_DIR, 'y_valid.csv'))

x_va_TF = np.minimum(x_va_df.values, 1.0).copy()
y_va_T = y_va_df.values[:,0].copy()

print("Validation data")
print("x_va_TF.shape: %s" % str(x_va_TF.shape))
print("y_va_T.shape : %s" % str(y_va_T.shape))
print("mean(y_va_T) : %.3f" % np.mean(y_va_T))


Validation data
x_va_TF.shape: (792, 7729)
y_va_T.shape : (792,)
mean(y_va_T) : 0.490


### Load test set 

In [8]:
x_te_df = pd.read_csv(os.path.join(DATA_DIR, 'x_test.csv.zip'))
y_te_df = pd.read_csv(os.path.join(DATA_DIR, 'y_test.csv'))

x_te_TF = np.minimum(x_te_df.values, 1.0).copy()
y_te_T = y_te_df.values[:,0].copy()

print("Heldout Test data")
print("x_te_TF.shape: %s" % str(x_te_TF.shape))
print("y_te_T.shape : %s" % str(y_te_T.shape))
print("mean(y_te_T) : %.3f" % np.mean(y_te_T))


Heldout Test data
x_te_TF.shape: (793, 7729)
y_te_T.shape : (793,)
mean(y_te_T) : 0.515


### Load vocabulary as a list of strings

In [9]:
vocab_list = x_tr_df.columns.tolist()


In [10]:
for word in vocab_list[:8]:
    print(word)
print("...")
for word in vocab_list[-8:]:
    print(word)


good
great
time
book
don't
work
i_have
read
...
never_get
i'd_like
loves_it
an_author
nomin
could_give
bad_but
gap


### Pack training and validation sets into big arrays (so we can use sklearn's hyperparameter search tools)

In [11]:
xall_LF = np.vstack([x_tr_NF, x_va_TF])
yall_L = np.hstack([y_tr_N, y_va_T])


In [12]:
valid_indicators_L = np.hstack([
    -1 * np.ones(y_tr_N.size), # -1 means never include this example in any test split
    0  * np.ones(y_va_T.size), #  0 means include in the first test split (we count starting at 0 in python)
    ])


In [13]:
# Create splitter object using Predefined Split
# Will be used later by all hyperparameter searches
my_splitter = sklearn.model_selection.PredefinedSplit(valid_indicators_L)


# Problem 1: Decision Trees

## 1A: Train a simple tree with depth 3

In [14]:
simple_tree = sklearn.tree.DecisionTreeClassifier(
    criterion='gini', 
    max_depth=3, min_samples_split=2, min_samples_leaf=1, 
    random_state=101)


### **Fit the tree** 

**TODO Train on the training set** in the next coding cell

In [15]:
simple_tree.fit(x_tr_NF, y_tr_N)


DecisionTreeClassifier(max_depth=3, random_state=101)

### **Figure 1: Print Tree** 

Use a helper function from the starter code

In [16]:
# call pretty_print_sklearn_tree on simple_tree, providing feature_names=vocab_list
pretty_print_sklearn_tree(simple_tree, feature_names=vocab_list)


The binary tree structure has 15 nodes.
- depth   0 has    1 nodes, of which    0 are leaves
- depth   1 has    2 nodes, of which    0 are leaves
- depth   2 has    4 nodes, of which    0 are leaves
- depth   3 has    8 nodes, of which    8 are leaves
The decision tree:  (Note: Y = 'yes' to above question; N = 'no')
Decision: X['great'] <= 0.50?
  Y Decision: X['excel'] <= 0.50?
    Y Decision: X['disappoint'] <= 0.50?
      Y Leaf: p(y=1 | this leaf) = 0.430 (4041 total training examples)
      N Leaf: p(y=1 | this leaf) = 0.114 (368 total training examples)
    N Decision: X['disappoint'] <= 0.50?
      Y Leaf: p(y=1 | this leaf) = 0.903 (277 total training examples)
      N Leaf: p(y=1 | this leaf) = 0.429 (14 total training examples)
  N Decision: X['return'] <= 0.50?
    Y Decision: X['bad'] <= 0.50?
      Y Leaf: p(y=1 | this leaf) = 0.745 (1413 total training examples)
      N Leaf: p(y=1 | this leaf) = 0.415 (142 total training examples)
    N Decision: X['movie'] <= 0.50?
    

## 1B : Find best Decision Tree with grid search

In [18]:
# Construct the default predictor
# Any hyperparameters here may be overridden by the hyperparameter grid

base_tree = sklearn.tree.DecisionTreeClassifier(
    criterion='gini',
    min_samples_split=2, min_samples_leaf=1)


In [19]:
tree_hyperparameter_grid_by_name = dict(
    max_depth=[2, 8, 32, 128],
    min_samples_leaf=[1, 3, 9],
    random_state = [101],
    )


**TODO Build the Grid Search** in the next coding cell

Follow the 1B instructions carefully

In [24]:
tree_grid_searcher = sklearn.model_selection.GridSearchCV(estimator=base_tree, param_grid=tree_hyperparameter_grid_by_name, scoring="balanced_accuracy", cv=my_splitter, return_train_score=True, refit=False, verbose=3, n_jobs=-1)


### Do the search!


In [25]:
start_time_sec = time.time()
tree_grid_searcher.fit(xall_LF, yall_L)
elapsed_time_sec = time.time() - start_time_sec


Fitting 1 folds for each of 12 candidates, totalling 12 fits


[CV 1/1] END max_depth=2, min_samples_leaf=9, random_state=101;, score=(train=0.640, test=0.634) total time=   1.7s
[CV 1/1] END max_depth=2, min_samples_leaf=3, random_state=101;, score=(train=0.640, test=0.634) total time=   2.4s
[CV 1/1] END max_depth=2, min_samples_leaf=1, random_state=101;, score=(train=0.640, test=0.634) total time=   2.6s
[CV 1/1] END max_depth=8, min_samples_leaf=9, random_state=101;, score=(train=0.712, test=0.696) total time=   4.1s
[CV 1/1] END max_depth=8, min_samples_leaf=1, random_state=101;, score=(train=0.728, test=0.699) total time=   4.8s
[CV 1/1] END max_depth=8, min_samples_leaf=3, random_state=101;, score=(train=0.723, test=0.698) total time=   4.9s
[CV 1/1] END max_depth=32, min_samples_leaf=9, random_state=101;, score=(train=0.812, test=0.720) total time=   8.5s
[CV 1/1] END max_depth=32, min_samples_leaf=3, random_state=101;, score=(train=0.875, test=0.735) total time=   8.5s
[CV 1/1] END max_depth=32, min_samples_leaf=1, random_state=101;, scor

### Build dataframe of results

Move the results of grid search into a nice pandas data frame.

In [26]:
if hasattr(tree_grid_searcher, 'cv_results_'):
    # Will be here if you called fit on tree_grid_searcher succesfully
    tree_search_results_df = pd.DataFrame(tree_grid_searcher.cv_results_).copy()
    print("Grid search of %3d configurations done after %6.1f sec" % (
        tree_search_results_df.shape[0], elapsed_time_sec))


Grid search of  12 configurations done after   12.8 sec


### Display search results

This block will make a pretty printed table of the results of your grid search

In [27]:
if hasattr(tree_grid_searcher, 'cv_results_'):
    pd.set_option('display.precision', 4)
    tree_keys = ['param_max_depth', 'param_min_samples_leaf']
    tree_search_results_df.sort_values(tree_keys, inplace=True)
    tree_search_results_df[tree_keys + ['mean_train_score', 'mean_test_score', 'rank_test_score', 'mean_fit_time']]


In [28]:
print("Printing a dict of the best hyperparameters")
try:
    print(tree_grid_searcher.best_params_)
except AttributeError:
    print("TODO")


Printing a dict of the best hyperparameters
{'max_depth': 128, 'min_samples_leaf': 1, 'random_state': 101}


### Build the best decision tree

**TODO Build the Best Tree on the training set** in the next coding cell

This is necessary so you have the specific best performing tree in your workspace.

Although you fit many trees in the search, they were not stored, so we need to recreate the best one.

Hint: Just feed the best hyperparameters as keyword args to construct the tree and fit it to the training data.

In [30]:
best_tree = base_tree.set_params(**tree_grid_searcher.best_params_)
best_tree.fit(x_tr_NF, y_tr_N)


DecisionTreeClassifier(max_depth=128, random_state=101)

### Interpret the best decision tree

In [25]:
pretty_print_sklearn_tree(best_tree, feature_names=vocab_list)


The binary tree structure has 1623 nodes.
- depth   0 has    1 nodes, of which    0 are leaves
- depth   1 has    2 nodes, of which    0 are leaves
- depth   2 has    4 nodes, of which    0 are leaves
- depth   3 has    8 nodes, of which    0 are leaves
- depth   4 has   16 nodes, of which    5 are leaves
- depth   5 has   22 nodes, of which    6 are leaves
- depth   6 has   32 nodes, of which   13 are leaves
- depth   7 has   38 nodes, of which   21 are leaves
- depth   8 has   34 nodes, of which   17 are leaves
- depth   9 has   34 nodes, of which   11 are leaves
- depth  10 has   46 nodes, of which   22 are leaves
- depth  11 has   48 nodes, of which   29 are leaves
- depth  12 has   38 nodes, of which   17 are leaves
- depth  13 has   42 nodes, of which   21 are leaves
- depth  14 has   42 nodes, of which   25 are leaves
- depth  15 has   34 nodes, of which   15 are leaves
- depth  16 has   38 nodes, of which   18 are leaves
- depth  17 has   40 nodes, of which   22 are leaves
- de

# Problem 2: Random forest

## 2A: Train a random forest with default settings

In [31]:
simple_forest = sklearn.ensemble.RandomForestClassifier(
    n_estimators=100,
    criterion='gini',
    max_features='sqrt',
    max_depth=3,
    min_samples_leaf=1,
    min_samples_split=2,
    random_state=101)


### Fit the forest

**TODO Train on the training set** in the next coding cell

In [32]:
simple_forest.fit(x_tr_NF, y_tr_N)


RandomForestClassifier(max_depth=3, random_state=101)

## 2B & Table 2: Feature Importances

### Table 2
**Sample Output** (Feel free to print all words and organize them in any software)

|**Important Words**|**Unimportant Words**|
|:-:|:-:|
|I1 |  U1  |
|I2 |  U2  |
|I3 |  U3  |
|I4 |  U4  |
|I5 |  U5  |
|I6 |  U6  |
|I7 |  U7  |
|I8 |  U8  |
|I9 |  U9  |
|I0 |  U0  |

In [47]:
np.flatnonzero(simple_forest.feature_importances_)



array([   1,    4,    5,   11,   12,   14,   17,   18,   33,   36,   37,
         40,   48,   53,   55,   58,   60,   62,   68,   70,   71,   72,
         78,   79,   82,   89,   90,   92,   93,   98,  100,  101,  103,
        113,  127,  130,  146,  147,  152,  167,  175,  180,  187,  189,
        194,  205,  206,  208,  222,  230,  233,  242,  247,  249,  266,
        267,  278,  280,  281,  282,  283,  289,  291,  294,  296,  300,
        315,  322,  327,  333,  336,  341,  350,  355,  361,  375,  381,
        387,  403,  411,  418,  419,  430,  434,  438,  440,  442,  443,
        444,  462,  466,  472,  481,  485,  486,  487,  493,  496,  505,
        513,  516,  522,  525,  530,  531,  543,  545,  554,  560,  561,
        570,  574,  578,  581,  584,  587,  599,  619,  622,  630,  646,
        649,  651,  652,  653,  661,  663,  665,  666,  672,  674,  680,
        681,  683,  704,  713,  715,  718,  727,  729,  763,  767,  769,
        773,  775,  776,  781,  785,  789,  800,  8

## 2C: Best Random Forest via grid search

Follow the instructions and using what you learn in 1B to finish this step.

This block might take 2-10 minutes. (Takes about 2 min on staff Macbook laptops.)

If yours runs significantly longer, try this out on Google Colab instead.

In [28]:
base_forest = sklearn.ensemble.RandomForestClassifier(
    n_estimators=100,
    criterion='gini',
    max_depth=16,
    min_samples_split=2,
    min_samples_leaf=1)


In [29]:
forest_hyperparameter_grid_by_name = dict(
    max_features=[3, 10, 33, 100, 333],
    max_depth=[16, 32],
    min_samples_leaf=[1],
    n_estimators=[100],
    random_state=[101],
    )


In [30]:
# TODO construct a GridSearchCV object like you did above.

forest_searcher = None 


### Do the search!

In [31]:
#TODO


### Build dataframe of results

In [32]:
#TODO


### Display search results

In [33]:
#TODO


### Build the best random forest using the best hyperparameters found in 2B 

This is necessary so you have the specific best performing forest in your workspace.

Train *only* on training set (do not merge train and valid)


In [34]:
#TODO


### Table 3: Comparison of methods on the bag-of-words to sentiment classification task.

Please report **balanced accuracy** on the train, valid, and test sets, to 3 digits of precision

**Sample Output** (Feel free to print all values and organize them by hand)

|**method**|**max depth**|**num trees**|**train BAcc**|**valid BAcc**|**test BAcc**|
|:-|:-:|:-:|:-:|:-:|:-:|
|simple Tree	| 1 | 1 | 0.123	|0.456	|0.890|
|best Tree	|1 | 1 | 0.123	|0.456	|0.890|
|simple RandomForest	|1 | 1 | 0.123	|0.456	|0.890|
|best RandomForest	|1 | 1 | 0.123	|0.456	|0.890|